<html>
    <div dir="rtl">
        این نوتبوک، روی دیتاست میراث جملات را بررسی می‌کند و ترتیب اجزاء جمله را بسته به POS tag شان یاد می‌گیرد.
        از این طریق می‌تواند بهترین جایگشت‌های ممکن برای کلمات را پیدا کند.
        <br>
        هر جمله‌ای که به مدل داده می‌شود، ابتدا tokenize می‌شود سپس pos tag هر توکن پیدا می‌شود.
        ساختار جمله از طریق این pos tag ها شناسایی می‌شود.
        سپس دنبال این ساختار شناسایی شده در داده‌های یادگیری (که از میراث استفاده شده) می‌گردد تا بهترین جایگشت این توکن‌ها را پیدا کند.
        تمام جایگشت‌های ممکن که احتمال خوبی برای درست بودن دارند به مدل‌های زبانی داده می‌شوند و به آن‌ها امتیاز تعلق می‌گیرد.
    </div>
</html>

In [48]:
%pip install hazm

Note: you may need to restart the kernel to use updated packages.


In [49]:
from hazm import POSTagger, Normalizer
from enum import Enum, auto
from hazm import SentenceTokenizer, WordTokenizer
from tqdm import tqdm
import itertools
tagger = POSTagger(model="resources/pos_tagger.model")

## تنظیمات

In [50]:
max_number_of_permutations = 30
min_acceptable_score_ngram = 10
min_acceptable_score_bert = 10

In [51]:
text = "رفته بودم خونه ی دوستم"

normalized_text = Normalizer().normalize(text)
tagged_text = tagger.tag(normalized_text.split())
print(tagged_text)

[('رفته', 'VERB'), ('بودم', 'VERB'), ('خونه\u200cی', 'NOUN,EZ'), ('دوستم', 'NOUN')]


In [52]:
"""
each sentence will be associated with a vector of len(Tag) elements, each element representing the number of words with that tag in the sentence
(N, Ne, V, Aj, Adv, Det, P, Conj, Pron, Num, Postp, Prep, Interj, Abbrev, Aux, Punc)
example: بابا آب داد
(2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0, 0, 0, 0, 0)
"""

class Tag(Enum):
    NOUN = auto()
    NE = auto()
    VERB = auto()
    ADJ = auto()
    ADV = auto()
    ADP = auto()
    DET = auto()
    P = auto()
    CONJ = auto()
    PRON = auto()
    NUM = auto()
    POSTP = auto()
    PREP = auto()
    INTJ = auto()
    ABBREV = auto()
    AUX = auto()
    PUNCT = auto()
    SCONJ = auto()
    CCONJ = auto()




"""
    dataset = {
        (1,0,2,...): {
            (N,V,N,...): 2,
            (N,N,V,...): 1,
        }, ...
    }
"""
def tag_dataset(resource='resources/Datasets/MirasText_sample.txt'):
    with open(resource, 'r') as f:
        text = f.read()
    sentences = SentenceTokenizer().tokenize(text)
    dataset = {}
    for sentece in tqdm(sentences):
        words = WordTokenizer().tokenize(sentece)
        tagged_words = tagger.tag(words)

        sentence_vector = [0] * len(Tag)
        for word, tag in tagged_words:
            pure_tag = tag.split(',')[0] # some tags are followed by ,EZ so we ignore it!
            tag_index = Tag[pure_tag].value - 1
            sentence_vector[tag_index] += 1

        sentence_parts_order = tuple([tag.split(',')[0] for _, tag in tagged_words])

        if tuple(sentence_vector) in dataset:
            if sentence_parts_order in dataset[tuple(sentence_vector)]:
                dataset[tuple(sentence_vector)][sentence_parts_order] += 1
            else:
                dataset[tuple(sentence_vector)][sentence_parts_order] = 1
        else:
            dataset[tuple(sentence_vector)] = {sentence_parts_order: 1}

    return dataset

<html>
    <div dir="rtl">
        برای این که فرایند ساخت tagged_dataset زمان بر است، از pickle کردن آن استفاده می‌کنیم که بتوانیم سریعا از روی فایل داده‌های train شده را بازیابی کنیم.
    </div>
</html>

In [53]:
import pickle

dataset_action = "load"

if dataset_action == "build":
    tagged_dataset = tag_dataset()
    dataset_action = "save"

if dataset_action == "save" and tagged_dataset:
    with open("resources/Datasets/trained_postag_orders.pickle", "wb") as f:
        pickle.dump(tagged_dataset, f)
        print("Saved dataset to pickle file!")
elif dataset_action == "load":
    with open("resources/Datasets/trained_postag_orders.pickle", "rb") as f:
        tagged_dataset = pickle.load(f)
        print("Loaded dataset from pickle file!")


Loaded dataset from pickle file!


In [54]:
# import pickle

# hamshahri_dataset = tag_dataset(resource='resources/Datasets/Hamshahri-Corpus.txt')
# with open("Datasets/hamshahri_trained_postag_orders.pickle", "wb") as f:
#         pickle.dump(tagged_dataset, f)

<html>
    <div dir="rtl">
        کاری که دوست داریم انجام بدهیم، این است که بتوانیم جملات را بر اساس اجزاء جمله‌شان دسته‌بندی کنیم.
        اما اگر بخواهیم به صورت دقیق تمام اجزاء را در نظر بگیریم کمی دشوار خواهد بود و به دیتاست خیلی بزرگ‌تری نیاز خواهیم داشت.
        برای همین باید بتوانیم کمی دسته‌بندی‌ها را جامع‌تر کنیم تا دسته‌های مختلف را در بر بگیرد.
        <br>
        برای حل این مشکل، چندین راه وجود دارد. یکی این که که 
    </div>
</html>

<html>
    <div dir="rtl">
    حال می‌خواهیم برای هر جمله، بهترین چینش‌ها را انتخاب کنیم. برای این کار باید POS tag های جمله را شناسایی کرده و به دنبال جایگشت‌های مطلوب آن در دیتاست ساخته شده بگردیم.
    </div>
</html>

In [97]:
def get_possible_permutations(sentence: str):
    words = WordTokenizer().tokenize(sentence)
    tagged_words = tagger.tag(words)
    permutations = []
    print(tagged_words)
    sentence_vector = [0] * len(Tag)
    for word, tag in tagged_words:
        pure_tag = tag.split(',')[0] # some tags are followed by ,EZ so we ignore it!
        tag_index = Tag[pure_tag].value - 1
        sentence_vector[tag_index] += 1

    vector_additions = [[0]*len(sentence_vector)] + [[0]*i + [1] + [0]*(len(sentence_vector)-i-1) for i in range(len(sentence_vector))]
    for addition, vector_addition in enumerate(vector_additions):
        vector = [x + y for x, y in zip(sentence_vector, vector_addition)]
        if tuple(vector) in tagged_dataset:
            proper_sentence_orders = tagged_dataset[tuple(vector)]
            sorted_sentence_orders = sorted(proper_sentence_orders.items(), key=lambda item: item[1], reverse=True)
            
            # build all permutations of the sentence
            all_permutations = list(itertools.permutations(tagged_words))
            for proper_order, _ in sorted_sentence_orders:
                if addition > 0:
                    the_order = tuple(filter(lambda x: x != Tag(addition).name , proper_order))
                else:
                    the_order = proper_order
                permutations += filter(lambda x: tuple([i[1].split(',')[0] for i in x]) == the_order, all_permutations)
                if len(permutations) >= max_number_of_permutations:
                    break
    return permutations[:max_number_of_permutations]

In [98]:
sample_text = "علی به من گفت"
possible_permutations = get_possible_permutations(sample_text)

for cp in possible_permutations:
    print(' '.join([i[0] for i in cp]))

[('علی', 'NOUN'), ('به', 'ADP'), ('من', 'PRON'), ('گفت', 'VERB')]
من به علی گفت
به من علی گفت


In [64]:
i = 0
for _ in tqdm(tagged_dataset.keys()):
    i += 1
print(i)

100%|██████████| 20076/20076 [00:00<00:00, 2079542.80it/s]

20076


In [99]:
sample_text = "در حال ایران . جمعیت است کاهش"
possible_permutations = get_possible_permutations(sample_text)

for cp in possible_permutations:
    print(' '.join([i[0] for i in cp]))

[('در', 'ADP'), ('حال', 'NOUN,EZ'), ('ایران', 'NOUN'), ('.', 'PUNCT'), ('جمعیت', 'NOUN'), ('است', 'VERB'), ('کاهش', 'NOUN')]
حال ایران در جمعیت کاهش است .
حال ایران در کاهش جمعیت است .
حال جمعیت در ایران کاهش است .
حال جمعیت در کاهش ایران است .
حال کاهش در ایران جمعیت است .
حال کاهش در جمعیت ایران است .
ایران حال در جمعیت کاهش است .
ایران حال در کاهش جمعیت است .
ایران جمعیت در حال کاهش است .
ایران جمعیت در کاهش حال است .
ایران کاهش در حال جمعیت است .
ایران کاهش در جمعیت حال است .
جمعیت حال در ایران کاهش است .
جمعیت حال در کاهش ایران است .
جمعیت ایران در حال کاهش است .
جمعیت ایران در کاهش حال است .
جمعیت کاهش در حال ایران است .
جمعیت کاهش در ایران حال است .
کاهش حال در ایران جمعیت است .
کاهش حال در جمعیت ایران است .
کاهش ایران در حال جمعیت است .
کاهش ایران در جمعیت حال است .
کاهش جمعیت در حال ایران است .
کاهش جمعیت در ایران حال است .
حال در ایران جمعیت کاهش است .
حال در ایران کاهش جمعیت است .
حال در جمعیت ایران کاهش است .
حال در جمعیت کاهش ایران است .
حال در کاهش ایران جمعیت است .
حال د

In [58]:
sample_text = "به مدرسه رفتم من."
possible_permutations = get_possible_permutations(sample_text)

for cp in possible_permutations:
    print(' '.join([i[0] for i in cp]))

[('به', 'ADP'), ('مدرسه', 'NOUN'), ('رفتم', 'VERB'), ('من', 'PRON'), ('.', 'PUNCT')]
من به مدرسه رفتم .
به من مدرسه رفتم .


In [100]:
sample_text = "رفتم به دانشگاه."
possible_permutations = get_possible_permutations(sample_text)

for cp in possible_permutations:
    print(' '.join([i[0] for i in cp]))

[('رفتم', 'VERB'), ('به', 'ADP'), ('دانشگاه', 'NOUN'), ('.', 'PUNCT')]
به دانشگاه رفتم .
دانشگاه به رفتم .
به دانشگاه رفتم .
به دانشگاه رفتم .
به دانشگاه رفتم .
به دانشگاه رفتم .
به دانشگاه رفتم .
به دانشگاه رفتم .
به دانشگاه رفتم .
به دانشگاه رفتم .
دانشگاه به رفتم .
